# 1.2_score_collection

The following score collection is focusing only on collecting the historical scores for 97 companies. Not every company has historical ESG data (roughly 500 companies when using yesg package). The scores are ultimately to be used as a comparison against the custom scoring created in Notebook 3.1. Hence, the list of 97 companies is a sample taken from the dataframe created in Notebook 3.1.

In [41]:
import yesg
import numpy as np
import joblib
import pandas as pd

In [42]:
df = joblib.load('../data/df_scores_test.pkl')

In [43]:
df

,E_weighted,G_weighted,S_weighted
ticker,,,
A,5.103045,7.743126,6.560172
AAPL,4.782933,7.692299,11.536030
ACGL,2.596256,4.995061,6.943322
AEP,18.229935,13.416611,18.009466
AIZ,3.679967,5.587310,4.273247
...,...,...,...
VZ,5.799947,12.644405,12.419938
WMT,6.088754,6.094006,7.251427
WRK,5.837043,5.884105,5.424793


The above dataframe is a sample of the larger dataframe created in Notebook 3.1 and contains a list of the scores created using LDA. The primary purpose of it is to get a list of tickers for which we can iterate through and search for historical ESG scores.

In [44]:
tick_list = df.index

In [45]:
len(tick_list)

97

The following code block iterates through the list of companies and grabs the historical scores. The ticker of the company is stored as the key in a dictionary, and the historical scores are stored as values. It is recommended you load the pickles instead of running the code block below.

In [46]:
# report_count = 0
# no_score = []
# tick_dict = {}
# tick_tracker = []
# for tick in tick_list:
#     if tick not in tick_tracker:
#         try:
#             df = yesg.get_historic_esg(tick)
#             if df is None:
#                 no_score.append(tick)
#             else:
#                 tick_tracker.append(tick)
#                 tick_dict[tick] = df
#                 report_count +=1 
#                 print(report_count)
#         except Exception as e:
#             print(tick)
#             print(e)
#     else:
#         continue

Using the above method, the maximum number of historical ESG scores that can be obtained is 93.

In [47]:
# import joblib
# tick_dict = joblib.dump(tick_dict,'../data/tick_dict_2.pkl')
# tick_tracker = joblib.dump(tick_tracker,'../data/tick_tracker_2.pkl')
# report_count = joblib.dump(report_count,'../data/report_count_2.pkl')

In [48]:
tick_dict = joblib.load('../data/tick_dict_2.pkl')
tick_tracker = joblib.load('../data/tick_tracker_2.pkl')
report_count = joblib.load('../data/report_count_2.pkl')

The below code turns the historical scores into a dataframe. 

In [49]:
df_fin = None
for i in list(range(0,len(list(tick_dict.values())))):
    df = pd.DataFrame(list(tick_dict.values())[i])
    df['tick'] = list(tick_dict.keys())[i]
    df_fin = pd.concat([df_fin,df], axis = 0)


In [50]:
df_fin.shape

(8732, 5)

The below code selects only the years where historical data is for 2021. 

In [51]:
df_fin = df_fin[df_fin.index.year == 2021]

Selecting only the month of January.

In [52]:
df_fin_jan = df_fin[df_fin.index.month.isin([1])]

In [53]:
df_fin_jan.head()

,Total-Score,E-Score,S-Score,G-Score,tick
Date,,,,,
2021-01-01,16.64,0.87,9.44,6.33,A
2021-01-01,16.46,0.10,7.55,8.82,AAPL
2021-01-01,26.56,2.15,11.03,13.37,ACGL
2021-01-01,34.44,19.33,9.83,5.28,AEP
2021-01-01,25.15,1.26,10.65,13.25,AIZ


In [54]:
df_fin_jan = df_fin_jan.reset_index()

In [55]:
df_fin_jan.drop(columns = 'Date', inplace = True)


In [56]:
df_fin_jan.set_index('tick', inplace=True)

In [57]:
df_fin_jan

,Total-Score,E-Score,S-Score,G-Score
tick,,,,
A,16.64,0.87,9.44,6.33
AAPL,16.46,0.10,7.55,8.82
ACGL,26.56,2.15,11.03,13.37
AEP,34.44,19.33,9.83,5.28
AIZ,25.15,1.26,10.65,13.25
...,...,...,...,...
VRTX,24.84,0.15,17.22,7.46
VTR,12.20,3.40,4.03,4.77
VZ,18.09,1.71,10.20,6.18


The above dataframe contains historical esg scores from January 2021 for 91 different companies.

The below section aims to rank each of the e, s and g scores based on which decile the scores fall into. This will allow us to compare my custom esg scores with the historical scores. Note that the higher the score, the greater the risk. Hence, to ensure that a lower score means that it a company falls into a higher decile, the inverse of the risk is used before taking the deciles. 

In [58]:
df_fin_jan['e_rank'] = pd.qcut(1/df_fin_jan['E-Score'],10, labels=False)
df_fin_jan['s_rank'] = pd.qcut(1/df_fin_jan['S-Score'],10, labels=False)
df_fin_jan['g_rank'] = pd.qcut(1/df_fin_jan['G-Score'],10, labels=False)

In [59]:
df_fin_jan

,Total-Score,E-Score,S-Score,G-Score,e_rank,s_rank,g_rank
tick,,,,,,,
A,16.64,0.87,9.44,6.33,8,4,4
AAPL,16.46,0.10,7.55,8.82,9,6,1
ACGL,26.56,2.15,11.03,13.37,6,2,0
AEP,34.44,19.33,9.83,5.28,0,4,7
AIZ,25.15,1.26,10.65,13.25,8,2,0
...,...,...,...,...,...,...,...
VRTX,24.84,0.15,17.22,7.46,9,0,3
VTR,12.20,3.40,4.03,4.77,6,8,8
VZ,18.09,1.71,10.20,6.18,7,3,5


In [60]:
joblib.dump(df_fin_jan, '../data/df_scores.pkl')

['../data/df_scores.pkl']

The above dataframe is to be used in Notebook 3.1 to compare the actual scores against the scores created via the novel scoring method. 